# Estimate radius

Can we improve generalization error by estimating the radius of the manifold, thereby eliminating asymmetric bias?

In [1]:
import numpy as np
from numpy import random
import tensorflow as tf

from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
sns.set(font_scale=1.3, rc={"lines.linewidth": 2.5})

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]='0'

In [3]:
root = '/mnt/fs2/bsorsch/manifold/embeddings/1k_objects/'
names = np.load(root + 'names.npy')
root_paths = np.load(root + 'root_paths.npy',allow_pickle=True)
names = names[:1000]
root_paths = root_paths[:1000]

model = 'resnet50'
save_dir = os.path.join(root, model+'_heldout/')

In [4]:
ilsvrc_root = '/mnt/fs2/bsorsch/manifold/embeddings/ILSVRC/'
centers_ilsvrc = np.load(ilsvrc_root + 'resnet50/centers.npy')

In [5]:
manifolds = np.load(save_dir + 'manifolds.npy')

In [6]:
lambdas = np.load(save_dir + 'lambdas.npy')
lambdas_ilsvrc = np.load(ilsvrc_root + 'resnet50/lambdas.npy')

ilsvrc_rp = np.load('../manifold_old/data/ILSVRC_root_paths.npy', allow_pickle=True)

In [10]:
from jax import jit
from jax import numpy as np

In [13]:
manifolds = np.array(manifolds)

In [ ]:
m = 5
n_avg = 2
P = manifolds.shape[1]

errs = []
errs_bias = []
Rhats = []
for a in tqdm(range(1000)):
    for b in tqdm(range(1000),leave=False):
        if a!=b:
            for _ in range(n_avg):
                Xa = manifolds[a]
                Xb = manifolds[b]
                perma = random.permutation(len(Xa))
                permb = random.permutation(len(Xb))

                xa,ya = np.split(Xa[perma],(m,))
                xb,yb = np.split(Xb[permb],(m,))
                w = (xa-xb).mean(0)
                mu = (xa+xb).mean(0)/2

                h = ya@w - w@mu
                err = (h<0).mean()
                errs.append(err)

                # Neighbor
                dists_a = ((xa.mean(0) - centers_ilsvrc)**2).sum(-1)
                dists_b = ((xb.mean(0) - centers_ilsvrc)**2).sum(-1)
                lamhat_a = lambdas_ilsvrc[np.argmin(dists_a)]
                lamhat_b = lambdas_ilsvrc[np.argmin(dists_b)]

                Ra = np.sqrt(lamhat_a*P)
                Rb = np.sqrt(lamhat_b*P)
                bias = ((Ra**2).mean() - (Rb**2).mean()) / 2 / m

                Rhats.append((Ra**2).sum())

                h = ya@w - w@mu + bias
                err_bias = (h<0).mean()
                errs_bias.append(err_bias)
errs = np.stack(errs)
errs_bias = np.stack(errs_bias)

In [ ]:
100*(1-errs_bias.mean()) - 100*(1-errs.mean())

In [ ]:
np.cos(2)

In [30]:
100*(1-errs.mean())

DeviceArray(98.62555, dtype=float32)

In [30]:
100*(1-errs_bias.mean())

DeviceArray(98.62555, dtype=float32)

In [21]:
print('Err: {:.3f} +/- {:.2f}'.format(1-errs.mean(), errs.std()))
print('Err bias: {:.3f} +/- {:.2f}'.format(1-errs_bias.mean(), errs_bias.std()))

Err: 0.987 +/- 0.04
Err bias: 0.987 +/- 0.04


In [63]:
print('Err: {:.3f} +/- {:.2f}'.format(1-errs.mean(), errs.std()))
print('Err bias: {:.3f} +/- {:.2f}'.format(1-errs_bias.mean(), errs_bias.std()))

Err: 0.922 +/- 0.17
Err bias: 0.929 +/- 0.16
